# Initial Cleaning/ setup procedures


#### _1. Import libraries and set path to pkl file_

In [4]:
import pandas as pd
import os
ROOT_DIR = os.path.realpath(os.path.join(os.getcwd(), '..'))
pkl_loc = os.path.join(ROOT_DIR, 'data', 'data','weathersmallraw.pkl')
pkl_loc

'C:\\Users\\paull\\OneDrive - Scandinavian Tobacco Group A S\\Documents\\GitHub\\Weatherman\\data\\data\\weathersmallraw.pkl'

### _2. Create dataframe and run basic eda_

In [5]:
df = pd.read_pickle(pkl_loc)
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3765246 entries, ('0CNUO', Timestamp('2022-01-01 00:00:00')) to ('ZYITU', Timestamp('2022-09-22 23:00:00'))
Data columns (total 11 columns):
 #   Column  Dtype  
---  ------  -----  
 0   temp    float64
 1   dwpt    float64
 2   rhum    float64
 3   prcp    float64
 4   snow    float64
 5   wdir    float64
 6   wspd    float64
 7   wpgt    float64
 8   pres    float64
 9   tsun    float64
 10  coco    float64
dtypes: float64(11)
memory usage: 330.7+ MB


In [6]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df.describe()

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
count,3765202.000,3765187.000,3765207.000,3744744.000,0.000,3765167.000,3765177.000,0.000,3765066.000,0.000,1614583.000
mean,9.246,3.645,71.106,0.116,NaN,184.294,13.762,NaN,1015.720,NaN,3.003
std,13.836,13.119,18.702,1.052,NaN,111.576,9.344,NaN,7.956,NaN,2.788
min,-43.000,-52.900,0.000,0.000,NaN,0.000,0.000,NaN,970.300,NaN,1.000
25%,-0.700,-5.700,58.000,0.000,NaN,91.000,7.600,NaN,1011.000,NaN,1.000
50%,12.200,6.200,73.000,0.000,NaN,191.000,13.000,NaN,1015.800,NaN,3.000
75%,20.300,14.800,86.000,0.000,NaN,280.000,18.400,NaN,1020.200,NaN,3.000
max,60.000,60.000,100.000,332.000,NaN,360.000,189.000,NaN,1058.700,NaN,25.000


In [7]:
df.shape

(3765246, 11)

### _3. Remove unneeded columns_


In [8]:
df.drop(columns=['snow', 'wpgt','tsun','coco'],inplace=True)
df.head()

temp    dwpt   rhum  prcp    wdir   wspd  \
station time                                                              
0CNUO   2022-01-01 00:00:00 -25.100 -28.100 76.000 0.000 271.000 12.600   
        2022-01-01 01:00:00 -25.700 -28.700 76.000 0.000 275.000 13.000   
        2022-01-01 02:00:00 -26.200 -29.300 75.000 0.000 273.000 12.600   
        2022-01-01 03:00:00 -26.100 -29.100 76.000 0.000 277.000 13.700   
        2022-01-01 04:00:00 -26.500 -29.600 75.000 0.000 275.000 13.300   

                                pres  
station time                          
0CNUO   2022-01-01 00:00:00 1020.800  
        2022-01-01 01:00:00 1021.500  
        2022-01-01 02:00:00 1022.500  
        2022-01-01 03:00:00 1022.600  
        2022-01-01 04:00:00 1023.200

In [13]:
df=df.reset_index()

In [14]:
df.groupby("station").count()

,time,temp,dwpt,rhum,prcp,wdir,wspd,pres
station,,,,,,,,
0CNUO,6352,6352,6352,6352,6350,6352,6352,6352
0CO7B,6352,6352,6352,6352,6350,6352,6352,6352
0FV1F,6352,6352,6352,6352,6350,6352,6352,6352
1J1PJ,6360,6360,6360,6360,6360,6360,6360,6360
1JWST,6360,6360,6360,6360,6360,6359,6360,6360
...,...,...,...,...,...,...,...,...
ZFZUV,6360,6360,6360,6360,6358,6360,6360,6360
ZNWZW,3658,3658,3658,3658,3638,3658,3658,3658
ZUQJS,3342,3342,3342,3342,3329,3342,3342,3331
